# Done by Mohamad Alkadri

A common problem in Numismatics, the study of currency, is to detect whether two coins have been struck by the same die. Striking a coin refers to pressing an image into the blank metal disc of the coin. To produce a coin, one must use two dies: one for the observe side (head) and another for the reverse side (tail). Our goal in this problem is to automatically predict given two coins, each as a pair of images representing observe and reverse sides, 1)whether the observe sides of those two coins have been struck by the same die, and 2) whether the reverse sides of those two coins have been struck by the same die.

To do this, we are providing you with a dataset consisting of 91 images of ancient coins of the Roman emperor Caracalla minted in Damascus. Each images depicts the observe and reverse sides of a coin. Also provided is an excel sheet that describes which observe sides and reverse sides of which coins have been struck by the same die. For example, given the following two rows from the excel sheet: 

1	O1	R1
2	O1	R2

This indicates that the observe sides of the two coins 1 and 2 have been struck by the same die, whereas their reverse sides were struck by different dies. 

You should build a deep learning model to solve the above problem. Once trained, your model should take two images representing two coins, and output whether their observe sides have been struck by the same die or not, and whether their reverse sides have been struck by the same die or not. You should submit your code as well as a write-up describing how you modeled the problem, what architecture and loss function you used, how did you split your data for training, validation, etc and finally some performance evaluation. Make sure you also submit your trained model so we can test it on other datasets. 

# EX2 Solution:

I will solve the problem as following: <ul>
    <li>1- Divide each image into two equal halfs, where the left side is the observe side and the right is the reverse side</li>
    <li>2- I will use Siamese neural networks to solve this task.I will train the network on the coins so that it give small distances for the coins that are struck by the same die and big distances for coins that are struck by different dies</li>
    <li>3- Finally, once the netwrok is trained well on the coin images, I will use one shot learning to tell if two halfs (images) are struck by the same die or not</li>
    
</ul>

# Read the labels

In [2]:
df=pd.read_csv("/home/cmps299/Desktop/ML/assi9/diestudy.csv",header=None)
df.columns=["id","head","tail"]
df[df["head"]=="O1"]

,id,head,tail
0,1,O1,R1
1,2,O1,R2
2,3,O1,R3
3,4,O1,R3
4,5,O1,R4
5,6,O1,R5


# Read all Images and cut them in half

In [98]:
# ''''' 
# I will do the following:
# 1- read the image coin
# 2- split the image into two halfs (head and coin)
# 3- place all similar heads in one folder and all similar tails in one folder
# ''''
path="/home/cmps299/Desktop/ML/assi9/Die Study Images/"
path_output="/home/cmps299/Desktop/ML/assi9/data/"

# for each coin image in the folder ....
for pic_path in glob.glob(path + "*"):
    #extract image id from it's absolute path:
    
    index=pic_path.rfind('/') # index of last character (/)
    pic_id = pic_path[index+1:] # to remove the absolute path 
    pic_id=int(re.sub('[^1-9]',"",pic_id)) # to remove .png .jpg ...
    
    # read the head and tail labels from diestudy.csv
    head_label=df[df["id"]==pic_id]["head"].values[0]
    tail_label=df[df["id"]==pic_id]["tail"].values[0]

    #read the image 
    img =   img_to_array(load_img(pic_path))
    height, width = img.shape[:-1]
    
    # Cut the image into two halfs (head and hail)
    width_cutoff = width // 2
    head = img[:, :width_cutoff,:]
    tail = img[:, width_cutoff:,:]
    
    # Save each half
    if not os.path.exists(path_output+head_label):
        os.makedirs(path_output+head_label)
        
    if not os.path.exists(path_output+tail_label):
        os.makedirs(path_output+tail_label)
    
    misc.imsave(path_output+head_label+"/"+str(pic_id)+"head.png", head)
    misc.imsave(path_output+tail_label+"/"+str(pic_id)+"tail.png", tail)
    


/home/cmps299/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/cmps299/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


# Apply data Augmnetation 

In [109]:
import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util

def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]

In [181]:
path_output="/home/cmps299/Desktop/ML/assi9/data/"
unique_id=95
# for each coin image in the folder ....
for folder in glob.glob(path_output + "*"):
    for pic in glob.glob(folder+"/" + "*"):
        #apply transformation to the image(s)
        img=cv2.imread(pic)
        for i in range (4):
            img1=random_rotation(img)
            img2=random_noise(img)
            img3=horizontal_flip(img)
            misc.imsave(folder+"/"+str(unique_id)+".png", img1)
            unique_id+=1
            misc.imsave(folder+"/"+str(unique_id)+".png", img2)
            unique_id+=1
            misc.imsave(folder+"/"+str(unique_id)+".png", img3)
            unique_id+=1

/home/cmps299/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  if sys.path[0] == '':
/home/cmps299/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  
/home/cmps299/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  app.launch_new_instance()


# Modeling: 

I took the implementation from this site: https://github.com/normandipalo/faceID_beta/blob/master/faceid_beta.ipynb 
However, with slight modification to fit my case 

In [3]:
import os
import sys

'''os.mkdir("faceid_train")
os.mkdir("faceid_val")'''
import numpy
import keras
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array


In [182]:
#define two functions: create_couple and create_wrong.
#create_couple: creates two images that are struck by the same die
# create_wrong: creates two images that are struck by different  die

In [4]:
path="/home/cmps299/Desktop/ML/assi9/data/"
#Input preprocessing.
#Here we create some functions that will create the input couple for our model, both correct and wrong couples. I created functions to have both depth-only input and RGBD inputs.
import numpy as np
import glob
#import matplotlib.pyplot as plt
from PIL import Image
import cv2

def create_couple(file_path):
    folder = np.random.choice(glob.glob(file_path + "*"))
    while folder == "datalab":
        folder = np.random.choice(glob.glob(file_path + "*"))
        #  print(folder)
    mat = np.zeros((480, 640), dtype='float32')
    i = 0
    j = 0
    depth_file = np.random.choice(glob.glob(folder + "/*.png"))
    mat = cv2.imread(depth_file)
    mat=img_to_array(load_img(depth_file,target_size=(200, 200)))
    mat = cv2.cvtColor(mat, cv2.COLOR_BGR2RGB)
#     mat_small = mat[350:550, 550:750]
    mat_small = mat
#     plt.imshow(mat_small)
#     plt.show()
    
    depth_file = np.random.choice(glob.glob(folder + "/*.png"))
    mat2 = cv2.imread(depth_file)
    mat2=img_to_array(load_img(depth_file,target_size=(200, 200)))

    mat2 = cv2.cvtColor(mat2, cv2.COLOR_BGR2RGB)
#     mat2_small = mat2[350:550, 550:750]
    mat2_small = mat2
#     plt.imshow(mat2_small)
#     plt.show()

    return np.array([mat_small, mat2_small])


print(create_couple(path).shape)

(2, 200, 200, 3)


In [5]:
def create_wrong(file_path):
    folder = np.random.choice(glob.glob(file_path + "*"))
    while folder == "datalab":
        folder = np.random.choice(glob.glob(file_path + "*"))
        
        
    depth_file = np.random.choice(glob.glob(folder + "/*.png"))
    mat = cv2.imread(depth_file)
    mat=img_to_array(load_img(depth_file,target_size=(200, 200)))

    mat = cv2.cvtColor(mat, cv2.COLOR_BGR2RGB)
#     mat_small = mat[350:550, 550:750]
    mat_small=mat

    folder2 = np.random.choice(glob.glob(file_path + "*"))
    while folder == folder2 or folder2 == "datalab":  # it activates if it chose the same folder
        folder2 = np.random.choice(glob.glob(file_path + "*"))
        
    depth_file = np.random.choice(glob.glob(folder2 + "/*.png"))
    mat2 = cv2.imread(depth_file)
    mat2=img_to_array(load_img(depth_file,target_size=(200, 200)))

    mat2 = cv2.cvtColor(mat2, cv2.COLOR_BGR2RGB)
    #     mat2_small = mat2[350:550, 550:750]
    mat2_small=mat2
    #plt.imshow(mat2_small)
    #plt.show()


    return np.array([mat_small, mat2_small])

print(create_wrong(path).shape)


(2, 200, 200, 3)


# Network crafting.
Now we create the network. We first manually create the *constrative loss*, then we define the network architecture starting from the SqueezeNet architecture, and then using it as a siamese-network for embedding faces into a manifold. (the network for now is very big and could be heavily optimized, but I just wanted to show a proof-of-concept)

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout, Lambda, ELU, concatenate, GlobalAveragePooling2D, Input, BatchNormalization, SeparableConv2D, Subtract, concatenate
from keras.activations import relu, softmax
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.optimizers import Adam, RMSprop, SGD
from keras.regularizers import l2
from keras import backend as K

In [7]:
def euclidean_distance(inputs):
    assert len(inputs) == 2, \
        'Euclidean distance needs 2 inputs, %d given' % len(inputs)
    u, v = inputs
    return K.sqrt(K.sum((K.square(u - v)), axis=1, keepdims=True))
        

def contrastive_loss(y_true,y_pred):
    margin=1.
    return K.mean((1. - y_true) * K.square(y_pred) + y_true * K.square(K.maximum(margin - y_pred, 0.)))
   # return K.mean( K.square(y_pred) )

In [8]:
def fire(x, squeeze=16, expand=64):
    x = Convolution2D(squeeze, (1,1), padding='valid')(x)
    x = Activation('relu')(x)
    
    left = Convolution2D(expand, (1,1), padding='valid')(x)
    left = Activation('relu')(left)
    
    right = Convolution2D(expand, (3,3), padding='same')(x)
    right = Activation('relu')(right)
    
    x = concatenate([left, right], axis=3)
    return x

In [9]:

img_input=Input(shape=(200,200,3))

x = Convolution2D(64, (5, 5), strides=(2, 2), padding='valid')(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

x = fire(x, squeeze=16, expand=16)

x = fire(x, squeeze=16, expand=16)

x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)


x = fire(x, squeeze=32, expand=32)

x = fire(x, squeeze=32, expand=32)

x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)


x = fire(x, squeeze=48, expand=48)

x = fire(x, squeeze=48, expand=48)

x = fire(x, squeeze=64, expand=64)

x = fire(x, squeeze=64, expand=64)

x = Dropout(0.2)(x)

x = Convolution2D(512, (1, 1), padding='same')(x)
out = Activation('relu')(x)


modelsqueeze= Model(img_input, out)

modelsqueeze.summary()




__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 98, 98, 64)   4864        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 98, 98, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 98, 98, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [10]:
im_in = Input(shape=(200,200,3))
#wrong = Input(shape=(200,200,3))

x1 = modelsqueeze(im_in)
#x = Convolution2D(64, (5, 5), padding='valid', strides =(2,2))(x)

#x1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x1)

"""
x1 = Convolution2D(256, (3,3), padding='valid', activation="relu")(x1)
x1 = Dropout(0.4)(x1)

x1 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1))(x1)

x1 = Convolution2D(256, (3,3), padding='valid', activation="relu")(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.4)(x1)

x1 = Convolution2D(64, (1,1), padding='same', activation="relu")(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.4)(x1)
"""



x1 = Flatten()(x1)

x1 = Dense(512, activation="relu")(x1)
x1 = Dropout(0.2)(x1)
#x1 = BatchNormalization()(x1)
feat_x = Dense(128, activation="linear")(x1)
feat_x = Lambda(lambda  x: K.l2_normalize(x,axis=1))(feat_x)


model_top = Model(inputs = [im_in], outputs = feat_x)

model_top.summary()

im_in1 = Input(shape=(200,200,3))
im_in2 = Input(shape=(200,200,3))

feat_x1 = model_top(im_in1)
feat_x2 = model_top(im_in2)


lambda_merge = Lambda(euclidean_distance)([feat_x1, feat_x2])


model_final = Model(inputs = [im_in1, im_in2], outputs = lambda_merge)

model_final.summary()

adam = Adam(lr=0.001)

sgd = SGD(lr=0.001, momentum=0.9)

model_final.compile(optimizer=adam, loss=contrastive_loss)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
model_1 (Model)              (None, 11, 11, 512)       252352    
_________________________________________________________________
flatten_1 (Flatten)          (None, 61952)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               31719936  
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
lambda_1 (Lambda)            (None, 128)               0         
Total para

# Learning phase.
We write the generators that will give our model batches of data to train on, then we run the training.

In [11]:
def generator(batch_size):
  
  while 1:
    X=[]
    y=[]
    switch=True
    for _ in range(batch_size):
   #   switch += 1
      if switch:
     #   print("correct")
        X.append(create_couple(path).reshape((2,200,200,3)))
        y.append(np.array([0.]))
      else:
     #   print("wrong")
        X.append(create_wrong(path).reshape((2,200,200,3)))
        y.append(np.array([1.]))
      switch=not switch
    X = np.asarray(X)
    y = np.asarray(y)
    XX1=X[0,:]
    XX2=X[1,:]
    yield [X[:,0],X[:,1]],y

In [12]:
def val_generator(batch_size):
  
  while 1:
    X=[]
    y=[]
    switch=True
    for _ in range(batch_size):
      if switch:
        X.append(create_couple(path).reshape((2,200,200,3)))
        y.append(np.array([0.]))
      else:
        X.append(create_wrong(path).reshape((2,200,200,3)))
        y.append(np.array([1.]))
      switch=not switch
    X = np.asarray(X)
    y = np.asarray(y)
    XX1=X[0,:]
    XX2=X[1,:]
    yield [X[:,0],X[:,1]],y

In [13]:
gen = generator(16)
val_gen = val_generator(4)

In [14]:
outputs = model_final.fit_generator(gen, steps_per_epoch=30, epochs=42, validation_data = val_gen, validation_steps=20)

Epoch 1/42
30/30 [==============================] - 38s 1s/step - loss: 0.2587 - val_loss: 0.4971
Epoch 2/42
30/30 [==============================] - 34s 1s/step - loss: 0.2506 - val_loss: 0.4966
Epoch 3/42
30/30 [==============================] - 34s 1s/step - loss: 0.2546 - val_loss: 0.4971
Epoch 4/42
30/30 [==============================] - 33s 1s/step - loss: 0.2527 - val_loss: 0.4959
Epoch 5/42
30/30 [==============================] - 33s 1s/step - loss: 0.2518 - val_loss: 0.4954
Epoch 6/42
30/30 [==============================] - 35s 1s/step - loss: 0.2550 - val_loss: 0.4938
Epoch 7/42
30/30 [==============================] - 33s 1s/step - loss: 0.2524 - val_loss: 0.4937
Epoch 8/42
30/30 [==============================] - 34s 1s/step - loss: 0.2521 - val_loss: 0.4897
Epoch 9/42
30/30 [==============================] - 34s 1s/step - loss: 0.2512 - val_loss: 0.4916
Epoch 10/42
30/30 [==============================] - 35s 1s/step - loss: 0.2569 - val_loss: 0.4903
Epoch 11/42
30/30 [

# Save  model's architecture and weights for a future use

In [64]:
from keras.models import model_from_json
output_source="/media/cmps299/Seagate Expansion Drive/model/"
# serialize model to JSON

# model_json = model_final.to_json()
# with open(output_source+"model.json", "w") as json_file:
#     json_file.write(model_json)

# serialize weights to HDF5
model_final.save_weights(output_source+"model.h5")
print("Saved model to disk")

# later...
# # load json and create model
# # json_file = open(output_source+'model.json', 'r')
# # loaded_model_json = json_file.read()
# # json_file.close()
# # model_final = model_from_json(loaded_model_json)

# # load weights into new model
# model_final.load_weights(output_source+"model.h5")
# print("Loaded model from disk")

Saved model to disk


In [28]:
#Check distance between two coins that are struck by the same die 
cop = create_couple(path)

model_final.evaluate([cop[0].reshape((1,200,200,3)), cop[1].reshape((1,200,200,3))], np.array([0.]))

1/1 [==============================] - 0s 24ms/step


0.00881088338792324

In [27]:
#Check distance between two coins that are struck by different  dies 
cop = create_wrong(path)

model_final.evaluate([cop[0].reshape((1,200,200,3)), cop[1].reshape((1,200,200,3))], np.array([0.]))

1/1 [==============================] - 0s 25ms/step


0.24336275458335876

We can notice the model's performance is good, because the model is giving small distance for coins that are struck by the same die and big distnace otherwise

I need to find a threshold for the distance, so that any distance below the threshold means that the coins are stuck by the same die and vice versa. <br>
Through random testing, I found out a threshould of (0.4) is a good threshold. However, a beter way to choose it, is by taking the mean distance between all coins that are struk by the same die, and the mean for the coins that are struck by different dies.

In [29]:
# do random testing to find a good threshold
similar_dis=0
diff_dis=0
for i in range (200):
    cop = create_couple(path)
    similar_dis+=model_final.evaluate([cop[0].reshape((1,200,200,3)), cop[1].reshape((1,200,200,3))], np.array([0.]),verbose=0)
    
    cop = create_wrong(path)
    diff_dis+=model_final.evaluate([cop[0].reshape((1,200,200,3)), cop[1].reshape((1,200,200,3))], np.array([0.]),verbose=0)
    
print(similar_dis/200,diff_dis/200)

0.0854896761525265 0.8982109160791151


In [30]:
threshold=0.4 # is a good threeshold. We can improve the model performance by choosing a better threshold

In [227]:
# writing a code that takes two images and tell whether their heads and tails are struck by the same die or not

In [63]:
coin1_path="/home/cmps299/Desktop/ML/assi9/Die Study Images/1.jpg"
coin2_path="/home/cmps299/Desktop/ML/assi9/Die Study Images/2.jpg"

head_total_right_hits=0
tail_total_righ_hits=0
n_sample=0
# n_sample=0
def predict_model(coin1_path,coin2_path,check_acc=False):
    global head_total_right_hits,tail_total_righ_hits,n_sample
    n_sample+=1
    # Read the image
    def split_image(path):

        img=img_to_array(load_img(path,target_size=(200, 400,3)))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        height, width = img.shape[:-1]

        # Cut the image in half
        width_cutoff = width // 2
        head = img[:, :width_cutoff,:]
        tail = img[:, width_cutoff:,:]

        return head,tail

    img1_head,img1_tail=split_image(coin1_path)
    img2_head,img2_tail=split_image(coin2_path)
    
    #extract img1 and img2 label
    index=coin1_path.rfind('/') # index of last character (/)
    pic_id = coin1_path[index+1:] # to remove the absolute path 
    img1_id=int(re.sub('[^1-9]',"",pic_id)) # to remove .png .jpg ...
    
    index=coin2_path.rfind('/') # index of last character (/)
    pic_id = coin2_path[index+1:] # to remove the absolute path 
    img2_id=int(re.sub('[^1-9]',"",pic_id)) # to remove .png .jpg ...
    
    # read the head and tail labels from diestudy.csv
    img1_head_label=df[df["id"]==img1_id]["head"].values[0]
    img1_tail_label=df[df["id"]==img1_id]["tail"].values[0]
    img2_head_label=df[df["id"]==img2_id]["head"].values[0]
    img2_tail_label=df[df["id"]==img2_id]["tail"].values[0]

    head_distance=model_final.evaluate([img1_head.reshape((1,200,200,3)), img2_head.reshape((1,200,200,3))], np.array([0.]),verbose=0)
    tail_distance=model_final.evaluate([img1_tail.reshape((1,200,200,3)), img2_tail.reshape((1,200,200,3))], np.array([0.]),verbose=0)
    
    if not check_acc:
        if head_distance < threshold:
            print("Head sides of the two coins 1 and 2 have been struck by the same die. Predicted:",True)
        else:
            print("Head sides of the two coins 1 and 2 have been struck by different dies. Predicted:",False)
        if tail_distance < threshold:
            print("Tail sides of the two coins 1 and 2 have been struck by the same die. Predicted:",True)
        else:
            print("Tail sides of the two coins 1 and 2 have been struck by different dies. Predicted:",False)
        
        print("Actual results for heads and tails: ",img1_head_label==img2_head_label,img1_tail_label==img2_tail_label,"Respectively")
        
        if img1_head_label==img2_head_label and head_distance < threshold or (img1_head_label!=img2_head_label and head_distance > threshold) :
            head_total_right_hits+=1
        if img1_tail_label==img2_tail_label and tail_distance < threshold or (img1_tail_label!=img2_tail_label and tail_distance > threshold):
            tail_total_righ_hits+=1


predict_model(coin1_path,coin2_path)
print("heads accuracy: ",head_total_right_hits/(n_sample))
print("tails accuracy: ",tail_total_righ_hits/(n_sample))

Head sides of the two coins 1 and 2 have been struck by the same die. Predicted: True
Tail sides of the two coins 1 and 2 have been struck by different dies. Predicted: False
Actual results for heads and tails:  True False Respectively
heads accuracy:  1.0
tails accuracy:  1.0


# Model Evaluation:

In [61]:
# we have 91 images. do the predictions on them
dir_path='/home/cmps299/Desktop/ML/assi9/Die Study Images'
path='/home/cmps299/Desktop/ML/assi9/Die Study Images/'
head_total_right_hits=0
tail_total_righ_hits=0
n_sample=0
i=0
for pic1 in glob.glob(path + "*"):
    j=0
    print(pic1)
    for pic2 in glob.glob(path + "*"):
        if j!=i: # so that we don't rea the same image twice 
            predict_model(pic1,pic2) 
        j+=1
    i+=1
    # (i) and (j) to prevent duplicate image
print("heads accuracy: ",head_total_right_hits/(n_sample))
print("tails accuracy: ",tail_total_righ_hits/(n_sample))


/home/cmps299/Desktop/ML/assi9/Die Study Images/88.bmp
/home/cmps299/Desktop/ML/assi9/Die Study Images/32.jpg
/home/cmps299/Desktop/ML/assi9/Die Study Images/62.jpg
/home/cmps299/Desktop/ML/assi9/Die Study Images/22.jpg
/home/cmps299/Desktop/ML/assi9/Die Study Images/68.jpg
/home/cmps299/Desktop/ML/assi9/Die Study Images/33.jpg
/home/cmps299/Desktop/ML/assi9/Die Study Images/20.jpg
/home/cmps299/Desktop/ML/assi9/Die Study Images/78.jpg
/home/cmps299/Desktop/ML/assi9/Die Study Images/17.jpg
/home/cmps299/Desktop/ML/assi9/Die Study Images/48.JPG
/home/cmps299/Desktop/ML/assi9/Die Study Images/67.jpg
/home/cmps299/Desktop/ML/assi9/Die Study Images/74.jpg
/home/cmps299/Desktop/ML/assi9/Die Study Images/73.jpg
/home/cmps299/Desktop/ML/assi9/Die Study Images/49.jpg
/home/cmps299/Desktop/ML/assi9/Die Study Images/10.jpg
/home/cmps299/Desktop/ML/assi9/Die Study Images/58.jpg
/home/cmps299/Desktop/ML/assi9/Die Study Images/30.jpg
/home/cmps299/Desktop/ML/assi9/Die Study Images/85.jpg
/home/cmps

In [62]:
print("heads accuracy: ",head_total_right_hits/(n_sample))
print("tails accuracy: ",tail_total_righ_hits/(n_sample))

heads accuracy:  0.5516483516483517
tails accuracy:  0.8105006105006105


heads accuracy:  0.5516483516483517 (55% of the predicted heads are predicted correct) <br>
tails accuracy:  0.8105006105006105 (81% of the predicted tails are predicted correct) <br>
The performance can be easily enhanced by picking a better threshold.

Rename the jupyter notebook to Assignment8_*netid*.ipynb (Assignment8_xyz01.ipynb) and upload it on Moodle no later than Wednesday, Nov 28 11:55 pm.